In [1]:
import sys
sys.path.append('../script/')
import os
from os.path import exists
from datetime import datetime
import json
import gc
from functools import partial
from collections import OrderedDict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
# from adabelief_pytorch import AdaBelief

import utils
import models
import train as trainer
DEVICE = "cuda"

# EPOCHS = 3000
MODELNAME = "Baseline1122"
# if not exists(MODELNAME):
#     os.makedirs(f"{MODELNAME}/tensorboard")
# now = datetime.now()
# now = str(now)[5:17].replace(" ", "_").replace(":", "")
# writer = SummaryWriter(log_dir=f"{MODELNAME}/tensorboard")

WEIGHT_PATHS = [f'{MODELNAME}/11-26_0045_fold{i}.pt' for i in range(5)]
PARAMS_PATH = f'{MODELNAME}/11-26_0045_params.json'

In [2]:
# TODO: クラスタごとに並べてConv1d
class BaseLine(nn.Module):
    def __init__(self, num_features, num_targets, num_layers=3, dropout=.2, hidden_size=256, activation="relu", batchnorm=True, weight_norm=True):
        super().__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(nn.utils.weight_norm(nn.Linear(num_features if len(layers)==0 else hidden_size, hidden_size, bias=(not batchnorm))))
            if batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout))
            if activation == "relu":
                layers.append(nn.ReLU())
            elif activation == "prelu":
                layers.append(nn.PReLU())
            else:
                raise RuntimeError(f'{activation} is not implemented')
        # layers.append(nn.utils.weight_norm(nn.Linear(hidden_size, num_targets)))
        layers.append(nn.Linear(hidden_size, num_targets))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [3]:
df = pd.read_csv("../input/folds/train.csv")
with open("../input/folds/targets", "r") as f:
    targets = f.read().split("\n")[:-1]
with open("../input/folds/features", "r") as f:
    features = f.read().split("\n")
with open(PARAMS_PATH) as f:
    params = json.load(f)

In [4]:
def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

---
## Predicting as usual

In [5]:
predictions = np.zeros((df.shape[0], len(targets)))
for fold in range(5):
    print(f'[Fold No.{fold:>3}] Predicting...', end='\r')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    va_idx = df[df.kfold == fold].index
    
    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    y_tr = train_df[targets].to_numpy()
    y_va = valid_df[targets].to_numpy()

    dataset_tr = utils.MoaDataset(x_tr, y_tr)
    loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
    dataset_va = utils.MoaDataset(x_va, y_va)
    loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)
    
    model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])
    
    weight = torch.load(WEIGHT_PATHS[fold], map_location=torch.device(DEVICE))
    weight = OrderedDict([(f'model.{k}', v) for k, v in weight.items()])
    model.load_state_dict(weight)
    
    model.eval()
    ps = []
    for ind, batch in enumerate(loader_va):
        ps.append(torch.sigmoid(model(batch["x"])).detach().cpu().numpy())
    ps = np.vstack(ps)
    predictions[va_idx] += ps
print()

[Fold No.  4] Predicting...


In [6]:
print(f'CV score               : {log_loss_metric(df[targets].values, predictions):.6}')
predictions_ = predictions.copy()
predictions_ = np.clip(predictions_,0.0005,0.999)
predictions_[df["cp_type_ctl_vehicle"]==1] = 0
print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score               : 0.016269
CV score w/ postprocess: 0.0161582


## Predicting w/ dropout ensemble

In [11]:
sg = nn.Sigmoid()
N_TRIALS = 100
predictions = np.zeros((df.shape[0], len(targets)))
for seed in range(N_TRIALS):
    for fold in range(5):
        print(f'[TRIAL No. {seed:>3}] [Fold No.{fold:>3}] Predicting...', end='\r')
        train_df = df[df.kfold != fold].reset_index(drop=True)
        valid_df = df[df.kfold == fold].reset_index(drop=True)
        va_idx = df[df.kfold == fold].index

        x_tr = train_df[features].to_numpy()
        x_va = valid_df[features].to_numpy()

        y_tr = train_df[targets].to_numpy()
        y_va = valid_df[targets].to_numpy()

        dataset_tr = utils.MoaDataset(x_tr, y_tr)
        loader_tr = torch.utils.data.DataLoader(dataset_tr, batch_size=512, num_workers=2, pin_memory=True)
        dataset_va = utils.MoaDataset(x_va, y_va)
        loader_va = torch.utils.data.DataLoader(dataset_va, batch_size=512, num_workers=2, pin_memory=True)

        model = BaseLine(num_features=x_tr.shape[1], num_targets=y_tr.shape[1], **params['nn_params'])

        weight = torch.load(WEIGHT_PATHS[fold], map_location=torch.device(DEVICE))
        weight = OrderedDict([(f'model.{k}', v) for k, v in weight.items()])
        model.load_state_dict(weight)

        model.train()
        ps = []
        for ind, batch in enumerate(loader_va):
            ps.append(torch.sigmoid(model(batch["x"])).detach().cpu().numpy())
        ps = np.vstack(ps)
        predictions[va_idx] += ps

[TRIAL No.   0] [Fold No.  4] Predicting...
[TRIAL No.   1] [Fold No.  4] Predicting...
[TRIAL No.   2] [Fold No.  4] Predicting...
[TRIAL No.   3] [Fold No.  4] Predicting...
[TRIAL No.   4] [Fold No.  4] Predicting...
[TRIAL No.   5] [Fold No.  4] Predicting...
[TRIAL No.   6] [Fold No.  4] Predicting...
[TRIAL No.   7] [Fold No.  4] Predicting...
[TRIAL No.   8] [Fold No.  4] Predicting...
[TRIAL No.   9] [Fold No.  4] Predicting...
[TRIAL No.  10] [Fold No.  4] Predicting...
[TRIAL No.  11] [Fold No.  4] Predicting...
[TRIAL No.  12] [Fold No.  4] Predicting...
[TRIAL No.  13] [Fold No.  4] Predicting...
[TRIAL No.  14] [Fold No.  4] Predicting...
[TRIAL No.  15] [Fold No.  4] Predicting...
[TRIAL No.  16] [Fold No.  4] Predicting...
[TRIAL No.  17] [Fold No.  4] Predicting...
[TRIAL No.  18] [Fold No.  4] Predicting...
[TRIAL No.  19] [Fold No.  4] Predicting...
[TRIAL No.  20] [Fold No.  4] Predicting...
[TRIAL No.  21] [Fold No.  4] Predicting...
[TRIAL No.  22] [Fold No.  4] Pr

In [8]:
print(f'CV score               : {log_loss_metric(df[targets].values, predictions):.6}')
predictions_ = predictions.copy()
predictions_ = np.clip(predictions_,0.0005,0.999)
predictions_[df["cp_type_ctl_vehicle"]==1] = 0
print(f'CV score w/ postprocess: {log_loss_metric(df[targets].values, predictions_):.6}')

CV score               : 0.0163489
CV score w/ postprocess: 0.0162451
